# Assignment 2A, Part 1: Evaluation

You are given two sample files, `data/sample_ranking.csv` and `data/sample_qrels.csv`, to test your solution.

This notebook is to be used for evaluating the rankings generated in [Part 2](2_Retrieval.ipynb) and [Part 3](3_Multifield_retrieval.ipynb).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
import os
%matplotlib notebook

In [242]:
BM25F_RANKING_FILE = "data/bm25_multifield.txt"  # file with the document rankings
BM25_RANKING_FILE = "data/bm25_singlefield.txt"  # file with the document rankings
LM_RANKING_FILE = "data/lm_singlefield.txt"  # file with the document rankings
MLM_RANKING_FILE = "data/lm_multifield.txt"  # file with the document rankings
QRELS_FILE = "data/qrels2.csv"  # file with the relevance judgments (ground truth)

**TODO**: Complete the function that calculates evaluation metrics for a given a ranking (`ranking`) against the ground truth (`gt`). It should return the results as a dictionary, where the key is the retrieval metric.

(Hint: see [Exercises #1 and #2 from Lecture 8](https://github.com/kbalog/uis-dat640-fall2019/tree/master/exercises/lecture_08).)

In [3]:
def eval_query(ranking, gt):
    """Calculates the ranking against the ground truth for a given query."""
    relevant, p10, ap, rr = 0, 0, 0, 0

    for i, document in enumerate(ranking):
        if document in gt:
            relevant += 1
            pi = relevant / (i+1)
            ap += pi
            
            if rr == 0:
                rr = 1 / (i+1)
            
            if i < 10:
                p10 += 1
    
    p10 /= 10
    ap /= len(gt)
    
    return {"P10": p10, "AP": ap, "RR": rr}

**TODO**: Complete the function that evaluates an output file, which contains rankings for a set of queries. It is almost complete, you just need to add the computation of mean scores (over the entire query set).

In [4]:
def evaluate(gt_file, output_file):
    """Prints evaluation scores for each query as well as the means over the query set."""
    # load data from ground truth file
    gt = {}  # holds a list of relevant documents for each queryID
    with open(gt_file, "r") as fin:
        header = fin.readline().strip()
        if header != "queryID,docIDs":
            raise Exception("Incorrect file format!")
        for line in fin.readlines():
            qid, docids = line.strip().split(",")
            gt[qid] = docids.split()
            
    # load data from output file
    output = {}
    with open(output_file, "r") as fin:
        header = fin.readline().strip()
        if header != "QueryId,DocumentId":
            raise Exception("Incorrect file format!")
        for line in fin.readlines():
            qid, docid = line.strip().split(",")
            if qid not in output:
                output[qid] = []
            output[qid].append(docid)
    
    # evaluate each query that is in the ground truth
    #print("  QID  P@10   (M)AP  (M)RR")
    sum_p10, sum_ap, sum_rr = 0, 0, 0
    for qid in sorted(gt.keys()):
        res = eval_query(output.get(qid, []), gt.get(qid, []))
        #print("%5s %6.3f %6.3f %6.3f" % (qid, res["P10"], res["AP"], res["RR"]))
        
        sum_p10 += res['P10']
        sum_ap += res['AP']
        sum_rr += res['RR']
    
    sum_p10 /= len(gt)
    sum_ap /= len(gt)
    sum_rr /= len(gt)
    
    # print averages
    #print("%5s %6.3f %6.3f %6.3f" % ("ALL", sum_p10, sum_ap, sum_rr))
    return sum_p10, sum_ap, sum_rr

### Main

## BM25

In [250]:
evaluate(QRELS_FILE, BM25_RANKING_FILE)

(0.20888888888888885, 0.07925703029969224, 0.43078904681362495)

In [240]:
def BM25_gridsearch_results(k_start=1.0, k_stop=2.0, k_step=0.1, b_start=0.0, b_stop=1.1, b_step=0.1):
    parameters = []
    p10 = []
    ap = []
    rr = []
    
    k_range = np.arange(k_start, k_stop, k_step)
    b_range = np.arange(b_start, b_stop, b_step)
    grid = itertools.product(k_range, b_range)
    
    for k, b in grid:
        ranking_file = 'data/gridsearch_bm25_k{:.3}_b{:.3}.txt'.format(k, b)
        results = evaluate(QRELS_FILE, ranking_file)
        p10.append(results[0])
        ap.append(results[1])
        rr.append(results[2])
        parameters.append((k,b))
        print('k = {:.2} - b = {:.2} - results = {}'.format(k, b, results))

    print('max p10 = {:.3} for parameters k={:.2}, b={:.2}'.format(max(p10), *parameters[p10.index(max(p10))]))
    print('max ap = {:.3} for parameters k={:.2}, b={:.2}'.format(max(ap), *parameters[ap.index(max(ap))]))
    print('max rr = {:.3} for parameters k={:.2}, b={:.2}'.format(max(rr), *parameters[rr.index(max(rr))]))

In [249]:
BM25_gridsearch_results()

k = 1.0 - b = 0.0 - results = (0.1555555555555555, 0.05225484743420211, 0.3059748276414943)
k = 1.0 - b = 0.1 - results = (0.21111111111111108, 0.07869412303235443, 0.43085290832814216)
k = 1.0 - b = 0.2 - results = (0.19555555555555548, 0.07648315146250878, 0.3585711983470931)
k = 1.0 - b = 0.3 - results = (0.19999999999999993, 0.07541622266002715, 0.33700702898133594)
k = 1.0 - b = 0.4 - results = (0.20222222222222214, 0.07526704414212083, 0.3396380018634351)
k = 1.0 - b = 0.5 - results = (0.20666666666666655, 0.07509852973165941, 0.35723350390516806)
k = 1.0 - b = 0.6 - results = (0.21111111111111103, 0.07323071118292224, 0.34169951668205906)
k = 1.0 - b = 0.7 - results = (0.19333333333333333, 0.06922935284484588, 0.32871105083724617)
k = 1.0 - b = 0.8 - results = (0.1733333333333333, 0.06495081430155265, 0.30523302916943434)
k = 1.0 - b = 0.9 - results = (0.15111111111111114, 0.06118636476100855, 0.290810934893994)
k = 1.0 - b = 1.0 - results = (0.1444444444444445, 0.05713764708869

k = 1.9 - b = 0.2 - results = (0.18666666666666665, 0.07455112157735427, 0.3514482170035353)
k = 1.9 - b = 0.3 - results = (0.18666666666666665, 0.07337780089193949, 0.3298464502101333)
k = 1.9 - b = 0.4 - results = (0.1866666666666666, 0.07210124249631628, 0.34668539777933294)
k = 1.9 - b = 0.5 - results = (0.18888888888888886, 0.06920108663945036, 0.3456159997007972)
k = 1.9 - b = 0.6 - results = (0.19777777777777772, 0.06541838029794309, 0.31589995897295736)
k = 1.9 - b = 0.7 - results = (0.18666666666666665, 0.06125613212482048, 0.3146154349932346)
k = 1.9 - b = 0.8 - results = (0.1622222222222222, 0.057369084983203755, 0.30789424551196415)
k = 1.9 - b = 0.9 - results = (0.14444444444444443, 0.051347771898307824, 0.280463533900999)
k = 1.9 - b = 1.0 - results = (0.12666666666666668, 0.04600151715152645, 0.2668848693950415)
max p10 = 0.211 for parameters k=1.0, b=0.1
max ap = 0.0793 for parameters k=1.2, b=0.1
max rr = 0.433 for parameters k=1.1, b=0.0


## BM25F

In [239]:
evaluate(QRELS_FILE, BM25F_RANKING_FILE)

(0.2155555555555555, 0.0802043268629494, 0.36363355944523296)

In [107]:
def BM25F_gridsearch_results(w_start = 0.0, w_stop = 0.3, w_step = 0.1, k_start=1.0, k_stop=1.4, k_step=0.1, b_content_start=0.2, b_content_stop=1.1, b_title_start=0.2, b_title_stop=1.1, b_step=0.1):
    parameters = []
    p10 = []
    ap = []
    rr = []
    
    w_range = np.arange(w_start, w_stop, w_step)
    k_range = np.arange(k_start, k_stop, k_step)
    b_content_range = np.arange(b_content_start, b_content_stop, b_step)
    b_title_range = np.arange(b_title_start, b_title_stop, b_step)
    grid = itertools.product(w_range, k_range, b_content_range, b_title_range)
    
    for w, k, bc, bt in grid:
        ranking_file = 'data/gridsearch_BM25F_w{:.3}_k{:.3}_bc{:.3}_bt{:.3}.txt'.format(w, k, bc, bt)
        if os.path.isfile(ranking_file):
            results = evaluate(QRELS_FILE, ranking_file)
            print(ranking_file)
            p10.append(results[0])
            ap.append(results[1])
            rr.append(results[2])
            parameters.append((w,k,bc,bt))
            print('w = {:.2} - k = {:.2} - bc = {:.2} - bt = {:.2} - results = {}'.format(w, k, bc, bt, results))

    print('max p10 = {:.3} for w = {:.2} - k = {:.2} - bc = {:.2} - bt = {:.2}'.format(max(p10), *parameters[p10.index(max(p10))]))
    print('max ap = {:.3} for w = {:.2} - k = {:.2} - bc = {:.2} - bt = {:.2}'.format(max(ap), *parameters[ap.index(max(ap))]))
    print('max rr = {:.3} for w = {:.2} - k = {:.2} - bc = {:.2} - bt = {:.2}'.format(max(rr), *parameters[rr.index(max(rr))]))

In [116]:
BM25F_gridsearch_results()

data/gridsearch_BM25F_w0.0_k1.0_bc0.2_bt0.2.txt
w = 0.0 - k = 1.0 - bc = 0.2 - bt = 0.2 - results = (0.13999999999999999, 0.052147643305662, 0.24643378226711563)
data/gridsearch_BM25F_w0.0_k1.0_bc0.2_bt0.3.txt
w = 0.0 - k = 1.0 - bc = 0.2 - bt = 0.3 - results = (0.19333333333333327, 0.07708385731980001, 0.35396399646399646)
data/gridsearch_BM25F_w0.0_k1.0_bc0.2_bt0.4.txt
w = 0.0 - k = 1.0 - bc = 0.2 - bt = 0.4 - results = (0.19333333333333327, 0.07708385731980001, 0.35396399646399646)
data/gridsearch_BM25F_w0.0_k1.0_bc0.2_bt0.5.txt
w = 0.0 - k = 1.0 - bc = 0.2 - bt = 0.5 - results = (0.19333333333333327, 0.07708385731980001, 0.35396399646399646)
data/gridsearch_BM25F_w0.0_k1.0_bc0.2_bt0.6.txt
w = 0.0 - k = 1.0 - bc = 0.2 - bt = 0.6 - results = (0.19333333333333327, 0.07708385731980001, 0.35396399646399646)
data/gridsearch_BM25F_w0.0_k1.0_bc0.2_bt0.7.txt
w = 0.0 - k = 1.0 - bc = 0.2 - bt = 0.7 - results = (0.19333333333333327, 0.07708385731980001, 0.35396399646399646)
data/gridsearch_BM

data/gridsearch_BM25F_w0.0_k1.1_bc0.3_bt0.9.txt
w = 0.0 - k = 1.1 - bc = 0.3 - bt = 0.9 - results = (0.19999999999999993, 0.07632835901695977, 0.3505618432708215)
data/gridsearch_BM25F_w0.0_k1.1_bc0.3_bt1.0.txt
w = 0.0 - k = 1.1 - bc = 0.3 - bt = 1.0 - results = (0.19999999999999993, 0.07632835901695977, 0.3505618432708215)
data/gridsearch_BM25F_w0.0_k1.1_bc0.4_bt0.2.txt
w = 0.0 - k = 1.1 - bc = 0.4 - bt = 0.2 - results = (0.20222222222222216, 0.07597185205176864, 0.35939976245757793)
data/gridsearch_BM25F_w0.0_k1.1_bc0.4_bt0.3.txt
w = 0.0 - k = 1.1 - bc = 0.4 - bt = 0.3 - results = (0.20222222222222216, 0.07597185205176864, 0.35939976245757793)
data/gridsearch_BM25F_w0.0_k1.1_bc0.4_bt0.4.txt
w = 0.0 - k = 1.1 - bc = 0.4 - bt = 0.4 - results = (0.20222222222222216, 0.07597185205176864, 0.35939976245757793)
data/gridsearch_BM25F_w0.0_k1.1_bc0.4_bt0.5.txt
w = 0.0 - k = 1.1 - bc = 0.4 - bt = 0.5 - results = (0.20222222222222216, 0.07597185205176864, 0.35939976245757793)
data/gridsearch_BM

data/gridsearch_BM25F_w0.0_k1.2_bc0.5_bt0.9.txt
w = 0.0 - k = 1.2 - bc = 0.5 - bt = 0.9 - results = (0.20222222222222216, 0.07425795064945473, 0.3673945237446125)
data/gridsearch_BM25F_w0.0_k1.2_bc0.5_bt1.0.txt
w = 0.0 - k = 1.2 - bc = 0.5 - bt = 1.0 - results = (0.20222222222222216, 0.07425795064945473, 0.3673945237446125)
data/gridsearch_BM25F_w0.0_k1.2_bc0.6_bt0.2.txt
w = 0.0 - k = 1.2 - bc = 0.6 - bt = 0.2 - results = (0.20222222222222222, 0.07278666092548393, 0.36436404716494986)
data/gridsearch_BM25F_w0.0_k1.2_bc0.6_bt0.3.txt
w = 0.0 - k = 1.2 - bc = 0.6 - bt = 0.3 - results = (0.20222222222222222, 0.07278666092548393, 0.36436404716494986)
data/gridsearch_BM25F_w0.0_k1.2_bc0.6_bt0.4.txt
w = 0.0 - k = 1.2 - bc = 0.6 - bt = 0.4 - results = (0.20222222222222222, 0.07278666092548393, 0.36436404716494986)
data/gridsearch_BM25F_w0.0_k1.2_bc0.6_bt0.5.txt
w = 0.0 - k = 1.2 - bc = 0.6 - bt = 0.5 - results = (0.20222222222222222, 0.07278666092548393, 0.36436404716494986)
data/gridsearch_BM

data/gridsearch_BM25F_w0.1_k1.1_bc0.3_bt0.2.txt
w = 0.1 - k = 1.1 - bc = 0.3 - bt = 0.2 - results = (0.21111111111111103, 0.07838022858114431, 0.3421680358918575)
data/gridsearch_BM25F_w0.1_k1.1_bc0.3_bt0.3.txt
w = 0.1 - k = 1.1 - bc = 0.3 - bt = 0.3 - results = (0.21111111111111103, 0.07838022858114431, 0.3421680358918575)
data/gridsearch_BM25F_w0.1_k1.1_bc0.3_bt0.4.txt
w = 0.1 - k = 1.1 - bc = 0.3 - bt = 0.4 - results = (0.21111111111111103, 0.07838022858114431, 0.3421680358918575)
data/gridsearch_BM25F_w0.1_k1.1_bc0.3_bt0.5.txt
w = 0.1 - k = 1.1 - bc = 0.3 - bt = 0.5 - results = (0.21111111111111103, 0.07838022858114431, 0.3421680358918575)
data/gridsearch_BM25F_w0.1_k1.1_bc0.3_bt0.6.txt
w = 0.1 - k = 1.1 - bc = 0.3 - bt = 0.6 - results = (0.21111111111111103, 0.07838022858114431, 0.3421680358918575)
data/gridsearch_BM25F_w0.1_k1.1_bc0.3_bt0.7.txt
w = 0.1 - k = 1.1 - bc = 0.3 - bt = 0.7 - results = (0.21111111111111103, 0.07838022858114431, 0.3421680358918575)
data/gridsearch_BM25F_

w = 0.1 - k = 1.2 - bc = 0.4 - bt = 0.9 - results = (0.21555555555555553, 0.07640376759809145, 0.3640275424071626)
data/gridsearch_BM25F_w0.1_k1.2_bc0.4_bt1.0.txt
w = 0.1 - k = 1.2 - bc = 0.4 - bt = 1.0 - results = (0.21555555555555553, 0.07640376759809145, 0.3640275424071626)
data/gridsearch_BM25F_w0.1_k1.2_bc0.5_bt0.2.txt
w = 0.1 - k = 1.2 - bc = 0.5 - bt = 0.2 - results = (0.21111111111111108, 0.07506214693253316, 0.38718677145144537)
data/gridsearch_BM25F_w0.1_k1.2_bc0.5_bt0.3.txt
w = 0.1 - k = 1.2 - bc = 0.5 - bt = 0.3 - results = (0.21111111111111108, 0.07506214693253316, 0.38718677145144537)
data/gridsearch_BM25F_w0.1_k1.2_bc0.5_bt0.4.txt
w = 0.1 - k = 1.2 - bc = 0.5 - bt = 0.4 - results = (0.21111111111111108, 0.07506214693253316, 0.38718677145144537)
data/gridsearch_BM25F_w0.1_k1.2_bc0.5_bt0.5.txt
w = 0.1 - k = 1.2 - bc = 0.5 - bt = 0.5 - results = (0.21111111111111108, 0.07506214693253316, 0.38718677145144537)
data/gridsearch_BM25F_w0.1_k1.2_bc0.5_bt0.6.txt
w = 0.1 - k = 1.2 

data/gridsearch_BM25F_w0.1_k1.3_bc0.6_bt0.6.txt
w = 0.1 - k = 1.3 - bc = 0.6 - bt = 0.6 - results = (0.19777777777777777, 0.07250431963070383, 0.3741848215736657)
data/gridsearch_BM25F_w0.1_k1.3_bc0.6_bt0.7.txt
w = 0.1 - k = 1.3 - bc = 0.6 - bt = 0.7 - results = (0.19777777777777777, 0.07250431963070383, 0.3741848215736657)
data/gridsearch_BM25F_w0.1_k1.3_bc0.6_bt0.8.txt
w = 0.1 - k = 1.3 - bc = 0.6 - bt = 0.8 - results = (0.19777777777777777, 0.07250431963070383, 0.3741848215736657)
data/gridsearch_BM25F_w0.1_k1.3_bc0.6_bt0.9.txt
w = 0.1 - k = 1.3 - bc = 0.6 - bt = 0.9 - results = (0.19777777777777777, 0.07250431963070383, 0.3741848215736657)
data/gridsearch_BM25F_w0.1_k1.3_bc0.6_bt1.0.txt
w = 0.1 - k = 1.3 - bc = 0.6 - bt = 1.0 - results = (0.19777777777777777, 0.07250431963070383, 0.3741848215736657)
data/gridsearch_BM25F_w0.2_k1.0_bc0.2_bt0.2.txt
w = 0.2 - k = 1.0 - bc = 0.2 - bt = 0.2 - results = (0.20222222222222222, 0.07760638829500519, 0.3563696760661393)
data/gridsearch_BM25F_

## LM

In [248]:
evaluate(QRELS_FILE, LM_RANKING_FILE)

(0.07777777777777779, 0.02055719068892328, 0.1398791625656122)

In [103]:
def LM_gridsearch_results(lambda_start = 0.0, lambda_stop = 1.1, lambda_step = 0.1, mu_start=1, mu_stop=10000, n_mu=5):
    smoothings = ['jelinek', 'dirichlet']
    lambda_range = np.arange(lambda_start, lambda_stop, lambda_step)
    mu_range = np.linspace(mu_start, mu_stop, n_mu)
    grid = [[smoothings[0], param] for param in lambda_range] + [[smoothings[1], param] for param in mu_range]
    
    parameters = []
    p10 = []
    ap = []
    rr = []
    
    for params in grid:
        ranking_file = 'data/gridsearch_lm_s{:.3}_param{:.3}.txt'.format(params[0], params[1])
        results = evaluate(QRELS_FILE, ranking_file)
        p10.append(results[0])
        ap.append(results[1])
        rr.append(results[2])
        parameters.append(params)
        print('smoothing = {} - param (lambda or mu) = {:.2} - results = {}'.format(params[0], params[1], results))
        
    print('max p10 = {:.3} for parameters smoothing={}, param={:.2}'.format(max(p10), *parameters[p10.index(max(p10))]))
    print('max ap = {:.3} for parameters smoothing={}, param={:.2}'.format(max(ap), *parameters[ap.index(max(ap))]))
    print('max rr = {:.3} for parameters smoothing={}, param={:.2}'.format(max(rr), *parameters[rr.index(max(rr))]))

In [244]:
LM_gridsearch_results()

smoothing = jelinek - param (lambda or mu) = 0.0 - results = (0.03777777777777779, 0.01061832445382231, 0.07018939208437998)
smoothing = jelinek - param (lambda or mu) = 0.1 - results = (0.03777777777777779, 0.01119342794654213, 0.07075936856655748)
smoothing = jelinek - param (lambda or mu) = 0.2 - results = (0.04000000000000001, 0.011724638820668518, 0.07135975379962636)
smoothing = jelinek - param (lambda or mu) = 0.3 - results = (0.04000000000000001, 0.01162791348813971, 0.0706021780420506)
smoothing = jelinek - param (lambda or mu) = 0.4 - results = (0.03555555555555556, 0.011648972703779755, 0.0689085551744243)
smoothing = jelinek - param (lambda or mu) = 0.5 - results = (0.037777777777777785, 0.011905466215185168, 0.07037761243148534)
smoothing = jelinek - param (lambda or mu) = 0.6 - results = (0.037777777777777785, 0.011932875851958978, 0.07027911089805168)
smoothing = jelinek - param (lambda or mu) = 0.7 - results = (0.037777777777777785, 0.01284790176725038, 0.07401673240700

## MLM

In [238]:
evaluate(QRELS_FILE, MLM_RANKING_FILE)

(0.1688888888888889, 0.054162261197333125, 0.35514118373476616)

In [105]:
def MLM_gridsearch_results(w_start = 0.0, w_stop = 1.1, w_step = 0.1, lambda_start = 0.0, lambda_stop = 1.1, lambda_step = 0.1, mu_start=1, mu_stop=10000, n_mu=8):
    smoothings = ['jelinek', 'dirichlet']
    lambda_range = np.arange(lambda_start, lambda_stop, lambda_step)
    w_range = np.arange(w_start, w_stop, w_step)
    mu_range = np.linspace(mu_start, mu_stop, n_mu)
    grid = [[smoothings[0], param] for param in lambda_range] + [[smoothings[1], param] for param in mu_range]
    grid = itertools.product(w_range, grid)
    
    parameters = []
    p10 = []
    ap = []
    rr = []
    
    for w, params in grid:
        ranking_file = 'data/gridsearch_mlm_w{:.3}_s{:.3}_param{:.3}.txt'.format(w, params[0], params[1])
        if os.path.isfile(ranking_file):
            results = evaluate(QRELS_FILE, ranking_file)
            p10.append(results[0])
            ap.append(results[1])
            rr.append(results[2])
            parameters.append([w,params])
            print('w = {:.2} - smoothing = {} - param = {:.2} - results = {}'.format(w, params[0], params[1], results))
        
    print('max p10 = {:.3} for parameters w={:.2}, smoothing={}, param={:.2}'.format(max(p10), parameters[p10.index(max(p10))][0], *parameters[p10.index(max(p10))][1]))
    print('max ap = {:.3} for parameters w={:.2}, smoothing={}, param={:.2}'.format(max(ap), parameters[p10.index(max(p10))][0], *parameters[ap.index(max(ap))][1]))
    print('max rr = {:.3} for parameters w={:.2}, smoothing={}, param={:.2}'.format(max(rr), parameters[p10.index(max(p10))][0], *parameters[rr.index(max(rr))][1]))

In [232]:
MLM_gridsearch_results()

w = 0.0 - smoothing = jelinek - param = 0.0 - results = (0.03777777777777779, 0.01061832445382231, 0.07018939208437998)
w = 0.0 - smoothing = jelinek - param = 0.1 - results = (0.03777777777777779, 0.01119342794654213, 0.07075936856655748)
w = 0.0 - smoothing = jelinek - param = 0.2 - results = (0.04000000000000001, 0.011724638820668518, 0.07135975379962636)
w = 0.0 - smoothing = jelinek - param = 0.3 - results = (0.04000000000000001, 0.01162791348813971, 0.0706021780420506)
w = 0.0 - smoothing = jelinek - param = 0.4 - results = (0.03555555555555556, 0.011648972703779755, 0.0689085551744243)
w = 0.0 - smoothing = jelinek - param = 0.5 - results = (0.037777777777777785, 0.011905466215185168, 0.07037761243148534)
w = 0.0 - smoothing = jelinek - param = 0.6 - results = (0.037777777777777785, 0.011932875851958978, 0.07027911089805168)
w = 0.0 - smoothing = jelinek - param = 0.7 - results = (0.037777777777777785, 0.01284790176725038, 0.07401673240700699)
w = 0.0 - smoothing = jelinek - par

w = 0.4 - smoothing = dirichlet - param = 1e+04 - results = (0.15111111111111106, 0.047745886534967745, 0.30895511100293505)
w = 0.5 - smoothing = jelinek - param = 0.0 - results = (0.046666666666666676, 0.012397644309201934, 0.09785219218666828)
w = 0.5 - smoothing = jelinek - param = 0.1 - results = (0.053333333333333344, 0.013177236273738006, 0.09918367329140104)
w = 0.5 - smoothing = jelinek - param = 0.2 - results = (0.06000000000000001, 0.013860215825749794, 0.1009383471947686)
w = 0.5 - smoothing = jelinek - param = 0.3 - results = (0.06444444444444446, 0.014175065176180583, 0.10691247711914947)
w = 0.5 - smoothing = jelinek - param = 0.4 - results = (0.06000000000000001, 0.014782517073660878, 0.11363286335132615)
w = 0.5 - smoothing = jelinek - param = 0.5 - results = (0.06444444444444446, 0.016437973557778107, 0.14147741154574078)
w = 0.5 - smoothing = jelinek - param = 0.6 - results = (0.06666666666666668, 0.018262978584113465, 0.16605575978494985)
w = 0.5 - smoothing = jelin

w = 0.9 - smoothing = dirichlet - param = 1e+04 - results = (0.15555555555555556, 0.047524827732794796, 0.3224329560915838)
w = 1.0 - smoothing = jelinek - param = 0.0 - results = (0.04666666666666667, 0.012429344153646651, 0.11345422672746729)
w = 1.0 - smoothing = jelinek - param = 0.1 - results = (0.053333333333333344, 0.012631553817899025, 0.11332034345338439)
w = 1.0 - smoothing = jelinek - param = 0.2 - results = (0.053333333333333344, 0.012841202328945187, 0.11495338113389372)
w = 1.0 - smoothing = jelinek - param = 0.3 - results = (0.053333333333333344, 0.013601796765243365, 0.13098905034044897)
w = 1.0 - smoothing = jelinek - param = 0.4 - results = (0.053333333333333344, 0.013677130532149076, 0.13116902351620274)
w = 1.0 - smoothing = jelinek - param = 0.5 - results = (0.053333333333333344, 0.013767886586695692, 0.1315462129284799)
w = 1.0 - smoothing = jelinek - param = 0.6 - results = (0.053333333333333344, 0.01391322968468816, 0.13386533337917927)
w = 1.0 - smoothing = jel